In [1]:
import plotly.express as px
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../summaries/curvature_dframe.csv", header= [0, 1], index_col=[0,1,2])

In [3]:
df.head()

metric                    curvature                      second_order_coeff_x  \
summary                      median       low       high               median   
fish date       condition                                                       
Fred 2020-09-23 floor      8.518469  5.532540  13.886612            -0.000228   
                well            NaN       NaN        NaN                  NaN   
     2020-09-25 floor      8.606701  4.782352  15.755039            -0.000152   
                well            NaN       NaN        NaN                  NaN   
     2020-09-27 floor      9.808012  5.366892  18.807091            -0.000181   

metric                                        second_order_coeff_y            \
summary                         low      high               median       low   
fish date       condition                                                      
Fred 2020-09-23 floor     -0.000471  0.000094             0.000158 -0.000109   
                well            NaN       NaN                  NaN       NaN   
     2020-09-25 floor     -0.000341  0.000088             0.000044 -0.000194   
                well            NaN       NaN                  NaN       NaN   
     2020-09-27 floor     -0.000411  0.000154             0.000075 -0.000165   

metric                              second_order_coeff_z                      
summary                        high               median       low      high  
fish date       condition                                                     
Fred 2020-09-23 floor      0.000369        -1.504932e-04 -0.000522  0.000145  
                well            NaN                  NaN       NaN       NaN  
     2020-09-25 floor      0.000238        -4.686214e-05 -0.000318  0.000170  
                well            NaN                  NaN       NaN       NaN  
     2020-09-27 floor      0.000301         1.663981e-07 -0.000242  0.000185

In [4]:
df = df.reset_index([0,1,2])
df_selected = df.loc[:, ["fish", "date", "condition", "curvature"]]
df_selected = df_selected.droplevel("summary", axis = 1)
df_selected.columns =  ["fish", "date", "condition", "median", "low", "high"]
df = df_selected.melt(id_vars = ["fish", "date", "condition"], var_name = "summary", value_name = "curvature")

array([-5, -4, -3, -2, -1,  1,  2,  3,  4,  5])

In [21]:
# add session instead of date
df["session_to_lesion"] = np.nan
df = df.sort_values(by=["fish","date"])
unique_fish = pd.unique(df.fish)
concat_ind = np.concatenate([np.arange(-5, 0), np.arange(1, 6)])
for f in unique_fish:
    dates = pd.unique(df.loc[df.fish == f, "date"]) # assuming these are sorted! careful
    #inds = np.arange(len(dates))
    for i, d in enumerate(dates):
        df.loc[(df.fish == f) & (df.date == d), "session_to_lesion"] = np.repeat(concat_ind[i], len(pd.unique(df.condition)) * len(pd.unique(df.summary)))


In [22]:
df['lesion_type'] = df["fish"].apply(lambda x: 'sham' if x in ["Greg", "Lazy", "Omar"] else 'lesion')

In [23]:
# todo: add means
fig = px.box(df[df.summary == "median"], x="session_to_lesion", 
                 y="curvature", color = "lesion_type", facet_col="condition",
                points = "all")
#fig = px.box(df, x="time", y="total_bill", points="all")

fig.show()

In [24]:
fig.write_image("../figs/curvature_summary.png")